This Python notebook accompies "Descent sets of cyclic permutations in types B and D." Algorithms from the article are implemented and theorems are checked for small values of n. Throughout, signed permutations in Bn are expressed in three ways: 

1) extended one-line notation as a single list [w(0),w(1),...,w(n)] 
2) cycle notation as a list containing the individual cycles, also encoded as lists [i,w(|i|),w(|w(i)|),...] 
3) for cyclic signed permutations in cycle notation, a single list [w(n),w(|w(n)|),...,n] or [w(n),w(|w(n)|),...,-n] 

In [1]:
import itertools
import numpy as np
import random 

# Generates tuples of positive and negative signs used for generating Bn
def gen_signs(n): 
    if n == 1:
        return list([[1],[-1]])
    else:
        signs = gen_signs(n-1)
        return [[1]+sign for sign in signs] + [[-1]+sign for sign in signs]

# Generates a list of the signed permutations Bn in the extended one-line notation
def gen_Bn(n):
    Bn = list()
    elements = list(range(1,n+1))
    Sn = itertools.permutations(elements)
    signs = gen_signs(n)
    for perm in Sn:
        for sign in signs:
            signed_perm = [0]+[i*j for i,j in zip(perm,sign)]
            Bn.append(signed_perm)
    return Bn

# Calculates descent set and descent statistic for a signed permutation in extended one-line notation
def Des(perm):
    n = len(perm)
    Des = list()
    for i in range(n-1):
        if perm[i]>perm[i+1]:
            Des.append(i)
    return Des
def des(perm):
    return len(Des(perm))

# Calculates descent set and descent statistic when ignoring the descent at the final possible position
def Modified_Des(perm):
    n = len(perm)
    return Des(perm[0:n-1])
def Modified_des(perm):
    return len(Modified_Des(perm))

In [2]:
# Generates a list of the cyclic signed permutations Cn
# Elements will have n or -n in the last position 
def gen_Cn(n):
    Cn = list()
    B = gen_Bn(n-1)
    for perm in B:
        long_cycle1 = perm.copy()+[n]
        long_cycle1.remove(0)
        Cn.append(long_cycle1)
        long_cycle2 = perm.copy()+[-n]
        long_cycle2.remove(0)
        Cn.append(long_cycle2)
    return Cn

# Counts the number of negatives in a list
def count_negs(L):
    return sum(np.sign(L) == -1)

# Generates a list of the cyclic permutations in Dn and in its complement
def gen_Dn_Dnc(n):
    Cn = gen_Cn(n)
    Dn = list()
    Dnc = list()
    for cycle in Cn:
        if count_negs(cycle) % 2 == 0:
            Dn.append(cycle)
        else:
            Dnc.append(cycle)
    return Dn,Dnc

# Converts cycle notation to extended one-line notation with two cases based on
# whether the input is a list of cycles or a single cycle 
def cycles_to_perm(cycles):
    if isinstance(cycles[0],list):
        n = sum([len(cycle) for cycle in cycles])
        perm = [0]*(n+1)
        for cycle in cycles:
            m = len(cycle)
            for i in range(m-1):
                perm[abs(cycle[i])]=cycle[i+1]
            perm[abs(cycle[m-1])]=cycle[0]
    else:
        n = len(cycles)
        perm = [0]*(n+1)
        for i in range(n-1):
            perm[abs(cycles[i])]=cycles[i+1]
        perm[abs(cycles[n-1])]=cycles[0]
    return perm

In [3]:
# Finds indices of the left-to-right maxima in a cycle 
def left_to_right_max(cycle):
    n = len(cycle)
    maxes = list([0])
    for i in range(1,n):
        if cycle[i]>cycle[maxes[-1]]:
            maxes.append(i)
    return maxes

# Splits a cycle into smaller cycles using left-to-right maxima, excluding the last cycle 
def split_cycle(cycle):
    n = len(cycle)
    maxes = left_to_right_max(cycle)
    return [cycle[maxes[i]:maxes[i+1]] for i in range(len(maxes)-1)]

# Replaces elements x and y in cycle notation with sgn(x)*y and sgn(y)*x
def swap(cycles,x,y):
    cycle_x = 0
    cycle_y = 0
    for h in range(len(cycles)):
        if x in cycles[h]:
            cycle_x = h
        if y in cycles[h]:
            cycle_y = h
    swapped_cycles = cycles.copy()
    swapped_cycles[cycle_x][cycles[cycle_x].index(x)] = np.sign(x)*abs(y)
    swapped_cycles[cycle_y][cycles[cycle_y].index(y)] = np.sign(y)*abs(x)
    return swapped_cycles

# Tests if min(x,y) is in the symmetric difference of descent sets for longer_perm and shorter_perm
# where x,y are consecutive elements in [n-1] and both perms are in extended one-line notation 
def P(longer_perm,shorter_perm,x,y):
    D1 = Des(longer_perm)
    D2 = Des(shorter_perm)
    n = len(shorter_perm)
    if min(x,y) > 0 and min(x,y) < n-1 and abs(x-y) == 1:
        if min(x,y) in D1 and min(x,y) not in D2:
            return True
        if min(x,y) in D2 and min(x,y) not in D1:
            return True
    return False

# Lowercase phi function mapping C_{n+1}^+ to B_n
# Gives output in both the cycle notation and one-line notation 
def phi(cycle):
    n = len(cycle)-1
    perm = cycles_to_perm(cycle)
    new_perm_cycle = split_cycle(cycle)
    new_perm = cycles_to_perm(new_perm_cycle)
    l = len(new_perm_cycle)
    for j in range(l):
        z = new_perm_cycle[j][-1]
        if P(perm,new_perm,abs(z),abs(z-1)) == True or P(perm,new_perm,abs(z),abs(z+1)) == True:
            if P(perm,new_perm,abs(z),abs(z-1)) == True and P(perm,new_perm,abs(z),abs(z+1)) == False:
                eps = -1
            elif P(perm,new_perm,abs(z),abs(z-1)) == False and P(perm,new_perm,abs(z),abs(z+1)) == True:
                eps = 1
            elif new_perm[abs(z+1)] > new_perm[abs(z-1)]:
                eps = 1
            else:
                eps = -1
            while P(perm,new_perm,abs(z),abs(z+eps)):
                x = z
                x_index = len(new_perm_cycle[j])-1
                for i in range(l):
                    if z+eps in new_perm_cycle[i]:
                        y = z+eps
                        y_cycle = i 
                        y_index = new_perm_cycle[i].index(y)
                    if -(z+eps) in new_perm_cycle[i]:
                        y = -(z+eps)
                        y_cycle = i 
                        y_index = new_perm_cycle[i].index(y)
                while P(perm,new_perm,abs(x),abs(y)):
                    new_perm_cycle = swap(new_perm_cycle,x,y)
                    new_perm = cycles_to_perm(new_perm_cycle)
                    if x_index != 0:
                        x = new_perm_cycle[j][x_index-1]
                        y = new_perm_cycle[y_cycle][y_index-1]
                        x_index = x_index - 1 
                        y_index = y_index - 1
                z = new_perm_cycle[j][-1]
    return new_perm_cycle,new_perm

In [4]:
# Changes all signs in a list, or list of lists
def negative(L):
    if isinstance(L[0],list):
        return [negative(l) for l in L]
    else:
        return [-1*i for i in L]
    
# Changes the sign of +1 or -1 in a list, or list of lists
def negative_1(L):
    if isinstance(L[0],list):
        return [negative_1(l) for l in L]
    else:
        n = len(L)
        new_L = [0]*n
        for i in range(n):
            if L[i] == 1 or L[i] == -1:
                new_L[i] = L[i]*(-1)
            else:
                new_L[i] = L[i]
        return new_L

# Capital Phi function mapping C_{n+1} to B_n
# Input is a single cycle of length n+1 with n+1 or -(n+1) in the final position
# Gives output in both the cycle notation and one-line notation 
def Phi(cycle):
    n = len(cycle)
    perm = cycles_to_perm(cycle)
    if n in cycle:
        new_perm_cycles,new_perm = phi(cycle)
    else: 
        new_perm_cycles,new_perm = phi(negative(cycle))
        new_perm_cycles = negative(new_perm_cycles)
        new_perm = negative(new_perm)
    if 0 not in Des(perm) and 0 in Des(new_perm):
        return negative_1(new_perm_cycles),negative_1(new_perm)
    elif 0 in Des(perm) and 0 not in Des(new_perm):
        return negative_1(new_perm_cycles),negative_1(new_perm)
    else:
        return new_perm_cycles,new_perm

In [5]:
# Brute-force check that Phi preserved descents at non-last positions
for n in range(2,10):
    counterexample_found = False
    C = gen_Cn(n)
    for cycle in C:
        perm = cycles_to_perm(cycle)
        Phi_perm_cycles,Phi_perm = Phi(cycle)
        if Modified_Des(perm) != Des(Phi_perm):
            print("Counterexample:",cycle,perm,Phi_perm_cycles,Phi_perm)
            counterexample_found = True
    if counterexample_found == False:
        print(f"No counterexamples for cyclic permutations in B{n}")

No counterexamples for cyclic permutations in B2
No counterexamples for cyclic permutations in B3
No counterexamples for cyclic permutations in B4
No counterexamples for cyclic permutations in B5
No counterexamples for cyclic permutations in B6
No counterexamples for cyclic permutations in B7
No counterexamples for cyclic permutations in B8
No counterexamples for cyclic permutations in B9


In [6]:
# Concatenates cycles in cycle notation into a single long cycle with n+1 at the end
def concat(cycles):
    new_cycle = []
    for cycle in cycles:
        new_cycle = new_cycle + cycle
    return new_cycle + [len(new_cycle)+1]

# Converts a permutation from extended one-line into canonical cycle notation 
def perm_to_canon_cycle(perm):
    n = len(perm)-1
    elements = set(perm[1:n+1])
    cycles = list()
    while elements != set():
        m = max(elements)
        new_cycle = [m]
        elements.remove(m)
        while perm[abs(new_cycle[-1])] != m:
            new_cycle.append(perm[abs(new_cycle[-1])])
            elements.remove(new_cycle[-1])
        cycles.append(new_cycle)
    return cycles[::-1]

# lowercase psi function that maps B_n to C_{n+1}^+
# Input a permutation in extended one-line notation and gives output in both cycle and one-line notation 
def psi(perm):
    new_perm_cycle = perm_to_canon_cycle(perm)
    new_perm = cycles_to_perm(concat(new_perm_cycle))
    l = len(new_perm_cycle)
    for j in range(l-1,-1,-1):
        z = new_perm_cycle[j][-1]
        if P(new_perm,perm,abs(z),abs(z-1)) == True or P(new_perm,perm,abs(z),abs(z+1)) == True:
            if P(new_perm,perm,abs(z),abs(z-1)) == True and P(new_perm,perm,abs(z),abs(z+1)) == False:
                eps = -1
            elif P(new_perm,perm,abs(z),abs(z-1)) == False and P(new_perm,perm,abs(z),abs(z+1)) == True:
                eps = 1
            elif new_perm[abs(z+1)] > new_perm[abs(z-1)]:
                eps = -1
            else:
                eps = 1
            while P(new_perm,perm,abs(z),abs(z+eps)):
                x = z
                x_index = len(new_perm_cycle[j])-1
                for i in range(l):
                    if z+eps in new_perm_cycle[i]:
                        y = z+eps
                        y_cycle = i 
                        y_index = new_perm_cycle[i].index(y)
                    if -(z+eps) in new_perm_cycle[i]:
                        y = -(z+eps)
                        y_cycle = i 
                        y_index = new_perm_cycle[i].index(y)
                while P(perm,new_perm,abs(x),abs(y)):
                    new_perm_cycle = swap(new_perm_cycle,x,y)
                    new_perm = cycles_to_perm(concat(new_perm_cycle))
                    if x_index != 0:
                        x = new_perm_cycle[j][x_index-1]
                        y = new_perm_cycle[y_cycle][y_index-1]
                        x_index = x_index - 1 
                        y_index = y_index - 1
                z = new_perm_cycle[j][-1]
    return concat(new_perm_cycle),new_perm

In [7]:
# Capital Psi function mapping B_n to D_{n+1}
# Gives output in cycle and one-line notation 
def Psi_D(perm):
    if perm[1] not in [-1,1]:
        if (count_negs(perm)%2) == 0:
            return psi(perm)
        else:
            new_perm_cycle,new_perm = psi(negative(perm))
            return negative(new_perm_cycle),negative(perm)
    elif perm[1] == 1:
        if (count_negs(perm)%2) == 0:
            return psi(perm)
        else:
            return psi(negative_1(perm))
    else:
        if (count_negs(perm)%2) == 0:
            new_perm_cycle,new_perm = psi(negative(negative_1(perm)))
            return negative(new_perm_cycle),negative(perm)
        else:
            new_perm_cycle,new_perm = psi(negative(perm))
            return negative(new_perm_cycle),negative(perm)

# Capital Psi function mapping B_n to the complement of D_{n+1}
# Gives output in cycle and one-line notation 
def Psi_Dc(perm):
    if perm[1] not in [-1,1]:
        if (count_negs(perm)%2) == 1:
            return psi(perm)
        else:
            new_perm_cycle,new_perm = psi(negative(perm))
            return negative(new_perm_cycle),negative(perm)
    elif perm[1] == 1:
        if (count_negs(perm)%2) == 1:
            return psi(perm)
        else:
            return psi(negative_1(perm))
    else:
        if (count_negs(perm)%2) == 1:
            new_perm_cycle,new_perm = psi(negative(negative_1(perm)))
            return negative(new_perm_cycle),negative(perm)
        else:
            new_perm_cycle,new_perm = psi(negative(perm))
            return negative(new_perm_cycle),negative(perm)

In [8]:
# Brute-force check that Psi_D and Psi_Dc are inverses of Phi on D_n and its complement
for n in range(2,10):
    counterexample_found = False
    D,Dc = gen_Dn_Dnc(n)
    for cycle in D:
        Phi_perm_cycles,Phi_perm = Phi(cycle)
        composed_cycle,composed_perm = Psi_D(Phi_perm)
        if cycle != composed_cycle: 
            print("Counterexample:",cycle,Phi_perm,composed_cycle)
            counterexample_found = True
    for cycle in Dc:
        Phi_perm_cycles,Phi_perm = Phi(cycle)
        composed_cycle,composed_perm = Psi_Dc(Phi_perm)
        if cycle != composed_cycle: 
            print("Counterexample:",cycle,Phi_perm,composed_cycle)
            counterexample_found = True
    if counterexample_found == False:
        print(f"No counterexamples for cyclic permutations in D{n} or its complement")

No counterexamples for cyclic permutations in D2 or its complement
No counterexamples for cyclic permutations in D3 or its complement
No counterexamples for cyclic permutations in D4 or its complement
No counterexamples for cyclic permutations in D5 or its complement
No counterexamples for cyclic permutations in D6 or its complement
No counterexamples for cyclic permutations in D7 or its complement
No counterexamples for cyclic permutations in D8 or its complement
No counterexamples for cyclic permutations in D9 or its complement


In [9]:
# Brute-force check for n=10 with modifications due to memory limitations 
# Note that this may take days to run
n = 10
counterexample_found = False 

Sn = itertools.permutations(list(range(1,n)))
signs = gen_signs(n)
for element in Sn:
    element = list(element)+[n]
    for sign in signs:
        cycle = [i*j for i,j in zip(element,sign)]
        perm = cycles_to_perm(cycle)
        Phi_perm_cycles,Phi_perm = Phi(cycle)
        if Modified_Des(perm) != Des(Phi_perm):
            print(f"Descent Counterexample for {perm} with image {Phi_perm}")
            counterexample_found = True 
        if count_negs(cycle) % 2 == 0:
            composed_cycle,composed_perm = Psi_D(Phi_perm)
        else:
            composed_cycle,composed_perm = Psi_Dc(Phi_perm)
        if cycle != composed_cycle: 
            print(f"Composition Counterexample for {cycle} with composition {composed_cycle}")
            
if counterexample_found == False:
    print("No counterexamples found")

No counterexamples found


In [10]:
# Generates a cyclic signed permutation randomly
def random_cyclic_perm(n):
    signs = random.choices([-1,1],k=n)
    perm = list(range(1,n))
    random.shuffle(perm)
    perm.append(n)
    return [i*j for i,j in zip(perm,signs)]

In [11]:
# Check theorems on N randomly generated signed cyclic permutations in C_n
n = 1000
N = 100000
counterexample_found = False 

for i in range(N):
    cycle = random_cyclic_perm(n)
    perm = cycles_to_perm(cycle)
    Phi_perm_cycles,Phi_perm = Phi(cycle)
    if Modified_Des(perm) != Des(Phi_perm):
        print(f"Descent Counterexample for {perm} with image {Phi_perm}")
        counterexample_found = True 
    if count_negs(cycle) % 2 == 0:
        composed_cycle,composed_perm = Psi_D(Phi_perm)
    else:
        composed_cycle,composed_perm = Psi_Dc(Phi_perm)
    if cycle != composed_cycle: 
        print(f"Composition Counterexample for {cycle} with composition {composed_cycle}")

if counterexample_found == False:
    print("No counterexamples found")

No counterexamples found
